In [1]:
import json
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from GoogleScraper import scrape_with_config, GoogleSearchError
from GoogleScraper.database import ScraperSearch, SERP, Link
import requests
import time

# 1. Configure Google Search

In [2]:
config = {
    'keyword': 'should site:askjelly.com',
    'num_pages_for_keyword': 100,
    'scrape_method': 'http',
    'do_caching': True,
    'output_filename': 'Results/google.json',
    'google_search_params': {'cr': 'countryUS'}
}

# 2. Get Google Results

In [3]:
try:
    search = scrape_with_config(config)
except GoogleSearchError as e:
    print(e)

# 3. Extract Links

In [4]:
links = []
for serp in search.serps:
    for link in serp.links:
        if link.link.startswith('https://askjelly.com'):
            links.append(link.link.strip())

In [5]:
#response = requests.get(links[0])
#f = open('response_test.txt','w')
#f.write(response.text)
#f.close()
#print(response.text)

# 4. Scrape Links

In [ ]:
scrape = []
for link in links:
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    question_f = soup.find("h2", {"data-reactid", "40"})
    answers_f = soup.find("div", { "class", "Question__answerCount___a5gPo type__subtitle___2S0EB"})
    topic_f = soup.find("div", {"class", "TopicPill__pill___XAbgr" })
    
    if question_f is not None:
        question = question_f.text
        topic = topic_f.text
        answers = -1
        if answers_f is not None:
            answers = answers_f.text
        
        scrape.append([link, question.strip(), answers, topic.strip()])
    else:
        scrape.append([link, None, None, None])
        

# 5. Output Results as CSV

In [7]:
scrapeDF = pd.DataFrame(scrape, columns=['Link', 'Question', 'Answers', 'Topic'])
scrapeDF.to_csv('Results/Jelly Scrape Results_Should w Topic' + time.strftime("%Y%m%d-%H%M%S") + '.csv', index=False)

# END